In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import time
import csv

In [ ]:
PATH = "C:\selenium\msedgedriver.exe"
course_path = r"https://course.ntu.edu.tw/search/dept"
output_path = "course.csv"
service = Service(PATH)
driver = webdriver.Edge(service=service)
r = requests.get(course_path)
soup = BeautifulSoup(r.text, 'html.parser')
driver.get("https://course.ntu.edu.tw/search/general")


In [ ]:
wait = WebDriverWait(driver, 10)
click_button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[3]/div/div/div/button")))
click_button.click()

In [ ]:
# 設定 DataFrame 列
info_list = []  # 用來儲存課程資訊的列表

wait = WebDriverWait(driver, 10)
courses = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//ul/li/div[2]/h2/a")))

last_height = driver.execute_script("return document.body.scrollHeight")

# 無限滾動，直到頁面到底部
while True:
    # 等待頁面加載並抓取所有課程資訊
    courses = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//ul/li")))

    # 逐一取得課程資訊
    for course in courses:
        try:
            # 課程名稱
            course_name = course.find_element(By.XPATH, ".//div[2]/h2/a").text.strip()

            # 老師
            teacher = course.find_element(By.XPATH, ".//div[2]/div").text.strip()

            # 學分數
            credits = course.find_element(By.XPATH, ".//div[3]/div[2]").text.strip()

            # 儲存到列表中
            info_list.append({"課程名稱": course_name, "老師": teacher, "學分數": credits})

            # 印出結果
            print(f"課程名稱: {course_name}, 老師: {teacher}, 學分數: {credits}")
        
        except Exception as e:
            print(f"❌ 無法獲取課程資訊: {e}")

    # 滾動頁面
    driver.execute_script("""
        window.scrollTo(0, document.body.scrollHeight);
    """)
    time.sleep(1)  # 等待 1 秒，讓頁面加載新內容

    # 檢查新的頁面高度
    new_height = driver.execute_script("return document.body.scrollHeight")
    # 如果頁面高度沒變，表示已經到底部，停止滾動
    if new_height == last_height:
        print("已經滾動到底部，停止滾動。")
        break

    last_height = new_height

# 將資料轉換為 DataFrame
info = pd.DataFrame(info_list, columns=["課程名稱", "老師", "學分數"])

# 印出資料
print(info)
info.head()


In [ ]:
# 儲存 DataFrame 到 CSV
output_path = "courses.csv"  # 設定檔案儲存路徑
info.to_csv(output_path, index=False, encoding='utf-8-sig')  # 使用 utf-8-sig 避免中文亂碼
print(f"CSV 檔案已儲存至: {output_path}")
